In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI()
cache_dir = LocalFileStore("../.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = TextLoader("./Files/chapter1.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)
retrival = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","you are a helpful assistance. Answer questions using only the following context. If you don't know the answer, just say you don't know. Don't make up: \n\n{context}"),
    ("human", "{question}"),
])
chain = {"context":retrival, "question":RunnablePassthrough()} | prompt | llm
# RunnablePassthrough : pass the input there
chain.invoke("describe victory mension")


AIMessage(content='Victory Mansion is a building where Winston resides in the novel. It is a dilapidated apartment building that is part of a grim and oppressive urban landscape. The apartment is described as rundown and lacking basic amenities.')